References:

https://stackoverflow.com/questions/54590363/create-tensorflow-dataset-from-image-local-directory

https://stackoverflow.com/questions/37340129/tensorflow-training-on-my-own-image

https://stackoverflow.com/questions/67055666/using-keras-apis-how-can-i-import-images-in-batches-with-exactly-k-instances-of

https://stackoverflow.com/questions/67288581/how-can-i-get-a-list-of-tensorflow-dataset-objects-from-a-directory-each-contai

https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices

In [2]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [3]:
!pwd

/content


**Download dataset from kaggle**



In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhilashrajan","key":"31d02119783186b2a4b6f25b69ac5188"}'}

In [5]:
! mkdir ~/.kaggle

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 32.2MB/s]
100% 270M/271M [00:01<00:00, 205MB/s]
100% 271M/271M [00:01<00:00, 156MB/s]
 95% 516M/543M [00:02<00:00, 270MB/s]
100% 543M/543M [00:02<00:00, 199MB/s]


In [9]:
#Unzipping the downloaded files

import os
import zipfile

for fzip in ['test1.zip', 'train.zip']:
  zip_ref = zipfile.ZipFile(fzip, 'r')
  zip_ref.extractall()
  zip_ref.close()

In [10]:
# There are two folders after unzipping

print(len(os.listdir('/content/train')))
print(len(os.listdir('/content/test1')))

25000
12500


In [11]:
# Defining folder structure to split the training and test data by label(cats and dogs)

try:
  parent_dir = '/tmp/cats-v-dogs'
  os.makedirs(parent_dir)
  training = os.path.join(parent_dir, "training")
  os.makedirs(training)
  testing = os.path.join(parent_dir, "testing")
  os.makedirs(testing)
  for folder in [training, testing]:
    for name in ['cats', 'dogs']:
      print(os.path.join(folder, name))
      os.makedirs(os.path.join(folder, name))
except OSError:
  pass

/tmp/cats-v-dogs/training/cats
/tmp/cats-v-dogs/training/dogs
/tmp/cats-v-dogs/testing/cats
/tmp/cats-v-dogs/testing/dogs


In [12]:
training

'/tmp/cats-v-dogs/training'

In [13]:
testing

'/tmp/cats-v-dogs/testing'

In [14]:
initial_list = os.listdir('/content/train')

In [15]:
len(initial_list)

25000

In [16]:
initial_list[:10]

['cat.8870.jpg',
 'dog.12413.jpg',
 'cat.3302.jpg',
 'dog.9739.jpg',
 'dog.5882.jpg',
 'cat.43.jpg',
 'cat.4321.jpg',
 'dog.1664.jpg',
 'dog.841.jpg',
 'cat.11685.jpg']

In [17]:
# Directories to seperate cats and dogs into two folders
os.makedirs('/tmp/PetImages/Cat/')
os.makedirs('/tmp/PetImages/Dog/')

In [18]:
# split the data into cat and Dog folder

for i in initial_list:
  if i[0:3] == 'cat':
    path = os.path.join('/content/train/',i )
    dest_path = os.path.join("/tmp/PetImages/Cat/",i )
    copyfile(path,dest_path)
  else:
    path = os.path.join('/content/train/',i )
    dest_path = os.path.join("/tmp/PetImages/Dog/",i )
    copyfile(path,dest_path)

In [19]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir("/tmp/PetImages/Dog/")))

12500
12500


In [20]:
# Split the data in the above two folders into training(cats and Dogs subfolders) and testing (cats and Dogs subfolders)

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    initial_list = os.listdir(SOURCE)
    rand_list = random.sample(initial_list, len(initial_list))
    train_folder_size = int(len(initial_list) * SPLIT_SIZE)
    for i in range(0, train_folder_size):
        path1 = os.path.join(SOURCE, rand_list[i])
        if os.path.getsize(path1) > 0:
            dest1 = os.path.join(TRAINING, rand_list[i])
            copyfile(path1,dest1)
    for i in range(train_folder_size,len(initial_list)):
        path2 = os.path.join(SOURCE, rand_list[i])
        if os.path.getsize(path2) > 0:
            dest2 = os.path.join(TESTING, rand_list[i])
            copyfile(path2,dest2)
            
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [21]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

11250
11250
1250
1250


**Steps to create seperate tf dataset objects by class label (Only doing it for the training data)**

In [23]:
os.listdir('/tmp/cats-v-dogs/training')

['cats', 'dogs']

In [29]:
base_path = '/tmp/cats-v-dogs/training'

In [48]:
# Function to create list (image_list_final) of lists where each list will have the filenames of each class and
# another list (label_list_final) of lists where each list will have the labels of the class 

def read_data():
  image_list_final = []
  label_list_final = []
  label_map_dict = {}
  count_label = 0
  for class_name in os.listdir(base_path):
    image_list = []
    label_list = []
    class_path = os.path.join(base_path, class_name)
    label_map_dict[class_name]=count_label
    for image_name in os.listdir(class_path):
      image_path = os.path.join(class_path, image_name)
      label_list.append(count_label)
      image_list.append(image_path)
    count_label += 1
    image_list_final.append(image_list)
    label_list_final.append(label_list)
  return image_list_final, label_list_final, label_map_dict

In [50]:
image_list_final, label_list_final, label_map_dict = read_data()

In [61]:
len(image_list_final)

2

In [89]:
len(image_list_final[0])

11250

In [90]:
image_list_final[0][0]

'/tmp/cats-v-dogs/training/cats/cat.8870.jpg'

In [63]:
len(label_list_final)

2

In [64]:
len(label_list_final[0])

11250

In [65]:
label_map_dict

{'cats': 0, 'dogs': 1}

In [72]:
# create the datasets where the dataset_list will have one dataset for images in each class

dataset_list = []
for i,j in zip(image_list_final,label_list_final):
    dataset_list.append(tf.data.Dataset.from_tensor_slices((tf.constant(i), tf.constant(j))))
    #dataset = dataset.shuffle(len(i))
    #dataset = dataset.repeat(epochs)
    #dataset = dataset.map(_parse_function).batch(batch_size)

In [87]:
len(dataset_list)

2

In [73]:
dataset_list[0]

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int32)>

In [74]:
for i in dataset_list[0].take(3):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/tmp/cats-v-dogs/training/cats/cat.8870.jpg'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'/tmp/cats-v-dogs/training/cats/cat.3302.jpg'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'/tmp/cats-v-dogs/training/cats/cat.4321.jpg'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [ ]:
# the below function to parse the filenames and labels in the dataset into arrays

def _parse_function(filename, label):
    image_string = tf.io.read_file(filename, "file_reader")
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

In [80]:
dataset = dataset_list[0].map(_parse_function)

In [81]:
dataset

<MapDataset shapes: ((None, None, 3), ()), types: (tf.float32, tf.int32)>

In [86]:
for i in dataset.take(1):
  print(i)

(<tf.Tensor: shape=(174, 173, 3), dtype=float32, numpy=
array([[[122., 140., 118.],
        [121., 139., 117.],
        [120., 135., 114.],
        ...,
        [ 94., 134.,  99.],
        [ 95., 135., 100.],
        [ 94., 134., 100.]],

       [[140., 155., 134.],
        [139., 154., 133.],
        [137., 152., 131.],
        ...,
        [ 94., 129.,  96.],
        [ 94., 129.,  96.],
        [ 93., 128.,  96.]],

       [[130., 143., 123.],
        [129., 142., 122.],
        [128., 142., 119.],
        ...,
        [102., 127.,  95.],
        [ 99., 124.,  92.],
        [ 98., 123.,  93.]],

       ...,

       [[132., 163., 121.],
        [130., 161., 119.],
        [130., 158., 117.],
        ...,
        [206., 192., 157.],
        [200., 186., 151.],
        [196., 182., 147.]],

       [[131., 164., 121.],
        [130., 163., 120.],
        [129., 160., 118.],
        ...,
        [211., 197., 162.],
        [210., 196., 161.],
        [207., 193., 158.]],

       [[132., 1